### Simulação Post-Predict

In [62]:
import findspark
findspark.init('/home/labdata/spark-2.2.1-bin-hadoop2.6')

In [63]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
spark = SparkSession\
            .builder\
            .appName('bank')\
            .getOrCreate()

In [64]:
from pyspark.sql import functions as F

In [65]:
prediction_file = "predictions/2018-08-10/predictions/part-00000-4bef2762-a8d6-4044-8175-e01bf8b372ed-c000.csv"

In [66]:
model_output = spark.read.csv(
    "hdfs://elephant:8020/user/labdata/" + prediction_file,
    header=False,
    sep=",",
    inferSchema=True
)

colnames = ['client_id', 'prediction', 'label', 'score']
model_output = model_output.selectExpr(*["{} as {}".format(model_output.columns[i], colnames[i]) for i in range(len(model_output.columns))])

. Qual a taxa de conversão para os grupos que o modelo previu compra vs não-compra?

In [67]:
results = model_output.groupBy('prediction').agg({"*": "count", "label": "avg"})
results = results.select(
    F.when(results['prediction']==1, 'yes').otherwise('no').alias('prediction'), 
    (results['count(1)']).alias('count'), F
    .format_number('avg(label)', 4).alias('convertion_rate')
)
results.show()

+----------+-----+---------------+
|prediction|count|convertion_rate|
+----------+-----+---------------+
|        no| 6896|         0.0566|
|       yes| 1342|         0.4001|
+----------+-----+---------------+



. Digamos que só temos mão para realizar 500 ligações (número arbitrário para fins de ilustração), quão melhor é a conversão se utilizando o modelo vs. escolher aleatóriamente? 

In [7]:
# aleatoriamente
random_sample = model_output.sample(
    withReplacement=False,
    fraction=0.0623,
    seed=420
)
cr_rand = 100*random_sample.groupBy().avg('label').collect()[0][0]

In [8]:
# top500
top500_sample = model_output.orderBy(F.desc('score')).limit(500)
cr_top = 100*top500_sample.groupBy().avg('label').collect()[0][0]

In [9]:
print(f"Taxa de Conversão Esperada:\n\tSem Modelo: {cr_rand:.2f}%\n\tCom Modelo: {cr_top:.2f}%")

Taxa de Conversão Esperada:
	Sem Modelo: 11.18%
	Com Modelo: 55.00%


. Em outro cenário, ligaríamos para todos os números disponíveis. Como muda o CAC ao ligar apenas para os recomendados pelo modelo?

In [51]:
# custo de cada ligação: R$ 17
call_cost = 17

In [52]:
convertion_full = model_output.groupBy('label').count()
convertion_full = {int(r['label']): r['count'] for r in convertion_full.collect()}

In [53]:
convertion_model = model_output.filter(model_output['prediction'] == 1).groupBy('label').count()
convertion_model = {int(r['label']): r['count'] for r in convertion_model.collect()}

In [55]:
cac_full = (convertion_full[0] + convertion_full[1])*call_cost / convertion_full[1]
cac_model = (convertion_model[0] + convertion_model[1])*call_cost / convertion_model[1]

In [61]:
print(f"CAC esperado:\n\tSem Modelo: R${cac_full:7.2f}\n\tCom Modelo: R${cac_model:7.2f}")

CAC esperado:
	Sem Modelo: R$ 151.07
	Com Modelo: R$  42.48
